### Epic 4 - Lookalikes
Als een key user kan ik voor een contact met weinig transacties een lookalike met veel transacties identificeren. Ik kan ook een clustering maken van contactpersonen die qua jobinhoud, type bedrijf, voorkeuren en (verwacht) gedrag.

As a key user, I can have a list of lookalikes for a contact that has little transactions, so I can use their recommendations. I can also make a clustering of contacts based on job content, company characteristics, preferences and (expected) behavior.

In [132]:
# Importing the necessary packages
import numpy as np                                  # "Scientific computing"
import scipy.stats as stats                         # Statistical tests

import pandas as pd                                 # Data Frame
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt                     # Basic visualisation
from statsmodels.graphics.mosaicplot import mosaic  # Mosaic diagram
import seaborn as sns   

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

**1. Data manipulation**

In [133]:
# Reading data
contacts = pd.read_csv('data/Contact.csv')
contact_functions = pd.read_csv('data/Contact functie.csv')
contact_function_descriptions = pd.read_csv('data/Functie.csv')

In [134]:
# Building contact profile
contacts = contacts.merge(contact_functions, how = 'left', left_on='crm_Contact_Contactpersoon', right_on='crm_ContactFunctie_Contactpersoon') # Merging contacts with their functions
contacts = contacts.merge(contact_function_descriptions, how = 'left', left_on='crm_ContactFunctie_Functie', right_on='crm_Functie_Functie') # Merging contacts with their function descriptions

# Removing rows and columns
col_to_remove = ['crm_Contact_Persoon_ID', 'crm_Contact_Status', 'crm_ContactFunctie_Contactpersoon', 'crm_ContactFunctie_Functie', 'crm_Functie_Functie']

contacts = contacts[contacts['crm_Contact_Status'] != 'Inactief']
contacts = contacts.drop(columns = col_to_remove)

# Showing dataframe
contacts

,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Voka_medewerker,crm_Functie_Naam
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,0,Bedrijfsleider
4,00005DF1-1FC9-E711-80EC-001DD8B72B62,C3C4F48F-1FC9-E711-80EC-001DD8B72B62,Partner,0,Echtgenote/Echtgenoot
8,0000A0A8-6BD1-E411-81DA-005056B06EB4,E4B21881-A267-E111-A00F-00505680000A,NaN,0,NaN
9,0000A149-846A-E111-B43A-00505680000A,4522C842-B467-E111-A00F-00505680000A,Gedelegeerd bestuurders N.V.,0,Bedrijfsleider
12,0000D2D1-C26D-E111-B43A-00505680000A,8FE57446-AC67-E111-A00F-00505680000A,Gedelegeerd bestuurder,0,Bedrijfsleider
...,...,...,...,...,...
1286188,FFFF0CA2-0674-E111-B43A-00505680000A,AF8EA9E8-F168-E111-B43A-00505680000A,Voorzitter van de raad van bestuur,0,Bedrijfsleider
1286190,FFFF2461-6762-EC11-8F8F-000D3A2E7738,00002561-6762-EC11-8F8F-000D3A2E7738,Werkend vennoot,0,Bedrijfsleider
1286192,FFFF437C-B673-E111-B43A-00505680000A,A11117AA-E668-E111-B43A-00505680000A,Zaakvoerder,0,Bedrijfsleider
1286198,FFFFB152-1EDC-E311-B4EE-005056B06EB4,0000B252-1EDC-E311-B4EE-005056B06EB4,Zaakvoerder,0,Contact Lidmaatschap


In [135]:
label_encoder = LabelEncoder()

# Encoding features to numeric values
contacts['crm_Contact_Account_encoded'] = label_encoder.fit_transform(contacts['crm_Contact_Account'])
contacts['crm_Contact_Functietitel_encoded'] = label_encoder.fit_transform(contacts['crm_Contact_Functietitel'])
contacts['crm_Functie_Naam_encoded'] = label_encoder.fit_transform(contacts['crm_Functie_Naam'])

# Showing dataframe
contacts

,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Voka_medewerker,crm_Functie_Naam,crm_Contact_Account_encoded,crm_Contact_Functietitel_encoded,crm_Functie_Naam_encoded
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,0,Bedrijfsleider,665,50485,3
4,00005DF1-1FC9-E711-80EC-001DD8B72B62,C3C4F48F-1FC9-E711-80EC-001DD8B72B62,Partner,0,Echtgenote/Echtgenoot,124521,34325,10
8,0000A0A8-6BD1-E411-81DA-005056B06EB4,E4B21881-A267-E111-A00F-00505680000A,NaN,0,NaN,145602,54462,69
9,0000A149-846A-E111-B43A-00505680000A,4522C842-B467-E111-A00F-00505680000A,Gedelegeerd bestuurders N.V.,0,Bedrijfsleider,44492,20404,3
12,0000D2D1-C26D-E111-B43A-00505680000A,8FE57446-AC67-E111-A00F-00505680000A,Gedelegeerd bestuurder,0,Bedrijfsleider,91510,20362,3
...,...,...,...,...,...,...,...,...
1286188,FFFF0CA2-0674-E111-B43A-00505680000A,AF8EA9E8-F168-E111-B43A-00505680000A,Voorzitter van de raad van bestuur,0,Bedrijfsleider,111588,49914,3
1286190,FFFF2461-6762-EC11-8F8F-000D3A2E7738,00002561-6762-EC11-8F8F-000D3A2E7738,Werkend vennoot,0,Bedrijfsleider,0,50232,3
1286192,FFFF437C-B673-E111-B43A-00505680000A,A11117AA-E668-E111-B43A-00505680000A,Zaakvoerder,0,Bedrijfsleider,102406,50485,3
1286198,FFFFB152-1EDC-E311-B4EE-005056B06EB4,0000B252-1EDC-E311-B4EE-005056B06EB4,Zaakvoerder,0,Contact Lidmaatschap,2,50485,8


**2. Getting similar contacts for a certain contact**

In [167]:
features = ['crm_Contact_Account_encoded', 'crm_Contact_Functietitel_encoded', 'crm_Contact_Voka_medewerker', 'crm_Functie_Naam_encoded']

# Extrating first contact 
first_contact = contacts.iloc[0][features]
first_contact = first_contact.values.reshape(1, -1) # Reshaping will make the dot product possible

# Calculating similarities
similarity = cosine_similarity(first_contact, contacts[features])

contacts['Similarity_with_contact'] = similarity[0]
contacts.sort_values(by = 'Similarity_with_contact', ascending = False)


,crm_Contact_Contactpersoon,crm_Contact_Account,crm_Contact_Functietitel,crm_Contact_Voka_medewerker,crm_Functie_Naam,crm_Contact_Account_encoded,crm_Contact_Functietitel_encoded,crm_Functie_Naam_encoded,Similarity_with_contact
0,00002D37-EF48-EB11-8119-001DD8B72B62,01002D37-EF48-EB11-8119-001DD8B72B62,Zaakvoerder,0,Bedrijfsleider,665,50485,3,1.000000
645043,80024633-D973-E111-B43A-00505680000A,01002F96-DC68-E111-B43A-00505680000A,Zaakvoerder,0,Bedrijfsleider,666,50485,3,1.000000
950872,BD178BEF-5E74-E111-B43A-00505680000A,00FFE0C4-1769-E111-B43A-00505680000A,Zaakvoerder,0,Bedrijfsleider,664,50485,3,1.000000
551445,6D6E4378-BF73-E111-B43A-00505680000A,00FFE0C4-1769-E111-B43A-00505680000A,Zaakvoerder,0,Bedrijfsleider,664,50485,3,1.000000
1286123,FFFBF127-23DC-E311-B4EE-005056B06EB4,00FCF127-23DC-E311-B4EE-005056B06EB4,Voorzitter-afgevaardigd bestuurder,0,Bestuurder,656,49943,4,1.000000
...,...,...,...,...,...,...,...,...,...
1252758,F963B4F5-48B5-E911-8104-001DD8B72B61,B7BFF30D-AB67-E111-A00F-00505680000A,Algemeen directeur,0,Politicus,116860,4,42,0.013205
3815,00C26419-D4B4-ED11-83FF-6045BD895CDC,D40381D1-B068-E111-B43A-00505680000A,Administratief medewerker,0,Politicus,134917,3,42,0.013193
767347,985679C1-92A7-ED11-AAD1-6045BD8952CE,FA344DB6-B268-E111-B43A-00505680000A,Aankoopdeskundige,0,Medewerker,159384,2,13,0.013184
1148798,E4CE16F2-CC5D-E811-80F0-001DD8B72B62,8F8CAEB3-C368-E111-B43A-00505680000A,"Junior Quality, Safety & Environment Manager",0,Verantwoordelijke Veiligheid,91298,1,65,0.013182
